In [6]:
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.stats import spearmanr

In [4]:
pd.set_option('display.expand_frame_repr', False)
np.random.seed(42)

In [9]:
PROJECT_DIR = Path('..', '..')
DATA_DIR = PROJECT_DIR / 'data'
DATA_DIR = 'C:/Users/86155/Machine Learning for Algorithmic Trading/data/assets.h5'

In [10]:
DATA_DIR

'C:/Users/86155/Machine Learning for Algorithmic Trading/data'

In [12]:
def get_backtest_data():
    """Combine chapter 7 lasso regression predictions
        with adjusted OHLCV Quandl Wiki data"""
    with pd.HDFStore('C:/Users/86155/Machine Learning for Algorithmic Trading/data/assets.h5') as store:
        prices = (store['quandl/wiki/prices']
                  .filter(like='adj')
                  .rename(columns=lambda x: x.replace('adj_', ''))
                  .swaplevel(axis=0))

    with pd.HDFStore('C:/Users/86155/Machine Learning for Algorithmic Trading/data.h5') as store:
        predictions = store['lasso/predictions']

    best_alpha = predictions.groupby('alpha').apply(lambda x: spearmanr(x.actuals, x.predicted)[0]).idxmax()
    predictions = predictions[predictions.alpha == best_alpha]
    predictions.index.names = ['ticker', 'date']
    tickers = predictions.index.get_level_values('ticker').unique()
    start = predictions.index.get_level_values('date').min().strftime('%Y-%m-%d')
    stop = (predictions.index.get_level_values('date').max() + pd.DateOffset(1)).strftime('%Y-%m-%d')
    idx = pd.IndexSlice
    prices = prices.sort_index().loc[idx[tickers, start:stop], :]
    predictions = predictions.loc[predictions.alpha == best_alpha, ['predicted']]
    return predictions.join(prices, how='right')

In [14]:
df = get_backtest_data()
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 190451 entries, ('AAPL', Timestamp('2014-12-09 00:00:00')) to ('LNG', Timestamp('2017-11-30 00:00:00'))
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   predicted  74054 non-null   float64
 1   open       190451 non-null  float64
 2   high       190451 non-null  float64
 3   low        190451 non-null  float64
 4   close      190451 non-null  float64
 5   volume     190451 non-null  float64
dtypes: float64(6)
memory usage: 14.2+ MB


In [15]:
df.to_hdf('backtest.h5', 'data')